## Day 10 - Pipes!

Pipes are one large continuous loop.

E.g. Square loop of pipe:
```
.....
.F-7.
.|.|.
.L-J.
.....
```

Find the single giant loop starting at S. How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?

In [1]:
with open("./example-4-steps.txt") as f:
    example_4_lines = [line.strip() for line in f.readlines()]
with open("./example-8-steps.txt") as f:
    example_8_lines = [line.strip() for line in f.readlines()]
with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]


example_4_lines

['.....', '.S-7.', '.|.|.', '.L-J.', '.....']

Idea:

 - 2D grid. Find that vector class from snake game.
 - Treat bends/pipes like instructions (i.e. + vector (1 0) for a straight up or something along those lines)

In [2]:
class Vector2D():
    def __init__(self, y, x):
        """
        np grid works like 
        (y,x)    (y, x+1)    (y, x+2)
        (y+1, x) (y+1, x+1)  (y+1, x+2)
        (y+2, x) .......
        """
        self.y = y
        self.x = x

    def __add__(self, v2):
        return Vector2D(self.y + v2.y, self.x + v2.x)
    
    def __sub__(self, v2):
        return Vector2D(self.y - v2.y, self.x - v2.x)

    def __iadd__(self, v2):
        self.x += v2.x
        self.y += v2.y
        return self
    
    def __isub__(self, v2):
        self.x -= v2.x
        self.y -= v2.y
        return self
    
    def __str__(self):
        return f'({self.y},{self.x})'
    
    def __eq__(self, v2):
        return (self.x==v2.x and self.y==v2.y)

    @property
    def as_tuple(self):
        return (self.y, self.x)

In [3]:
def create_grid_map(lines: list[str]) -> tuple[dict, tuple[int, int]]:
    """
    Takes in the input text (separated by line into a list)
    and notes the value at each location in a dict

    so key in like position (1,2) and get value at that location.

    USING SYSTEM:
        (y, x)
        where (0,0) is top left corner

    Returns dict map and also the starting position!
    """
    grid_map = {}
    starting_position = None
    for y_index, line in enumerate(lines):
        for x_index, val in enumerate(line):
            if val == "S":
                starting_position = (y_index, x_index)
            grid_map[(y_index, x_index)] = val

    if starting_position is None:
        raise ValueError("Didn't find starting position!")
    return grid_map, starting_position

- `|` is a vertical pipe connecting north and south.
- `-` is a horizontal pipe connecting east and west.
- `L` is a 90-degree bend connecting north and east.
- `J` is a 90-degree bend connecting north and west.
- `7` is a 90-degree bend connecting south and west.
- `F` is a 90-degree bend connecting south and east.
- `.` is ground; there is no pipe in this tile.
- `S` is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.

In [4]:
class Plumber():
    """
    Who else would be going through the pipes?!
    """
    UP = Vector2D(-1,0)
    DOWN = Vector2D(1,0)
    LEFT = Vector2D(0,-1)
    RIGHT = Vector2D(0,1)

    # yes this is dumb I couldn't be bothered to rewrite

In [5]:
def get_new_direction_from_instruction_and_relative_pos(instruction: str, relative_position: Vector2D) -> Vector2D:
    """
    relative_position means instruction_pos - current_pos
    """
    if instruction == "|":
        if relative_position == Plumber.UP:
            return relative_position
        elif relative_position == Plumber.DOWN:
            return relative_position
        else:
            raise ValueError("|")
    if instruction == "-":
        if relative_position == Plumber.LEFT:
            return relative_position
        elif relative_position == Plumber.RIGHT:
            return relative_position
        else:
            raise ValueError("-")
    if instruction == "L":
        if relative_position == Plumber.DOWN:
            return Plumber.RIGHT
        elif relative_position == Plumber.LEFT:
            return Plumber.UP
        else:
            raise ValueError("L")
    if instruction == "J":
        if relative_position == Plumber.DOWN:
            return Plumber.LEFT
        elif relative_position == Plumber.RIGHT:
            return Plumber.UP
        else:
            raise ValueError("J")
    if instruction == "7":
        if relative_position == Plumber.UP:
            return Plumber.LEFT
        elif relative_position == Plumber.RIGHT:
            return Plumber.DOWN
    if instruction == "F":
        if relative_position == Plumber.UP:
            return Plumber.RIGHT
        elif relative_position == Plumber.LEFT:
            return Plumber.DOWN
    
    raise ValueError(f"What are we doing with {instruction} here?")


In [6]:
import copy

class Plumber():
    """
    Who else would be going through the pipes?!
    """
    UP = Vector2D(-1,0)
    DOWN = Vector2D(1,0)
    LEFT = Vector2D(0,-1)
    RIGHT = Vector2D(0,1)

    def __init__(self, starting_position: Vector2D, current_dir: Vector2D, grid_map: dict):
        starting_position = copy.deepcopy(starting_position)
        self.pos: Vector2D = starting_position
        self.current_dir: Vector2D = current_dir
        self.grid_map: dict = grid_map
    
    def move(self, dir : Vector2D = None):
        """
        moves the Plumber in chosen direction.
        """

        if dir is None:
            relative_position_to_next_point = self.current_dir
            next_instruction = self.grid_map[(self.pos + self.current_dir).as_tuple]
            if next_instruction == "S":
                self.pos += self.current_dir
            else:
                next_direction = get_new_direction_from_instruction_and_relative_pos(
                    next_instruction, relative_position_to_next_point
                )
                self.pos += self.current_dir
                self.current_dir = next_direction

        else:
            # manually

            # Dont go backwards surely
            if self.current_dir == self.RIGHT and dir == self.LEFT:
                raise ValueError("Backwards!!!")
            if self.current_dir == self.LEFT and dir == self.RIGHT:
                raise ValueError("Backwards!!!")
            if self.current_dir == self.UP and dir == self.DOWN:
                raise ValueError("Backwards!!!")
            if self.current_dir == self.DOWN and dir == self.UP:
                raise ValueError("Backwards!!!")

            # Set new current direction
            self.current_dir = dir
            # Update position
            self.pos += self.current_dir

In [7]:
def find_first_moves(grid_map: dict, starting_position: Vector2D) -> list[tuple[Vector2D, Vector2D]]:
    """
    Returns a list of two tuples.
    In each the first value is where to move to then the second is the new current direction
    """
    assert grid_map[starting_position.as_tuple] == "S", f"No S in starting position {starting_position}? - {grid_map[starting_position.as_tuple]}"

    above_val = grid_map.get((starting_position + Plumber.UP).as_tuple)
    below_val = grid_map.get((starting_position + Plumber.DOWN).as_tuple)
    left_val = grid_map.get((starting_position + Plumber.LEFT).as_tuple)
    right_val = grid_map[(starting_position + Plumber.RIGHT).as_tuple]

    initial_direcitons = []
    if above_val is not None:
        try:
            direction = get_new_direction_from_instruction_and_relative_pos(above_val, Plumber.UP)
            initial_direcitons.append(
                (Plumber.UP, direction)
            )
        except:
            pass
    if below_val is not None:
        try:
            direction = get_new_direction_from_instruction_and_relative_pos(below_val, Plumber.DOWN)
            initial_direcitons.append(
                (Plumber.DOWN, direction)
            )
        except:
            pass
    if left_val is not None:
        try:
            direction = get_new_direction_from_instruction_and_relative_pos(left_val, Plumber.LEFT)
            initial_direcitons.append(
                (Plumber.LEFT, direction)
            )
        except:
            pass
    if right_val is not None:
        try:
            direction = get_new_direction_from_instruction_and_relative_pos(right_val, Plumber.RIGHT)
            initial_direcitons.append(
                (Plumber.RIGHT, direction)
            )
        except:
            pass

    assert len(initial_direcitons) == 2, initial_direcitons

    return tuple(initial_direcitons)

In [8]:
def part1(lines: list[str]) -> int:
    grid_map, starting_position_tuple = create_grid_map(lines)
    starting_position = Vector2D(y=starting_position_tuple[0], x=starting_position_tuple[1])

    # INITIAL SET UP
    first_moves_list = find_first_moves(grid_map, starting_position)
    move1, dir1 = first_moves_list[0]
    move2, dir2 = first_moves_list[1]
    plumber1 = Plumber(starting_position, move1, grid_map)
    plumber2 = Plumber(starting_position, move2, grid_map)
    plumber1.move(move1)
    plumber1.current_dir = dir1
    plumber2.move(move2)
    plumber2.current_dir = dir2

    steps_plumber_1 = 1
    # REST OF THE LOOP!
    while True:
        plumber1.move()
        steps_plumber_1 += 1
        # print(plumber1.pos, grid_map[plumber1.pos.as_tuple], plumber1.current_dir)
        if grid_map[plumber1.pos.as_tuple] == "S":
            break

    steps_plumber_2 = 1
    # REST OF THE LOOP!
    while True:
        plumber2.move()
        steps_plumber_2 += 1
        if grid_map[plumber2.pos.as_tuple] == "S":
            break
    
    assert (answer:= int(steps_plumber_1 / 2)) == int(steps_plumber_2 / 2)

    return answer

assert part1(example_4_lines) == 4
assert part1(example_8_lines) == 8
part1(input_lines)

6903

Surprised that just worked tbh

**Part 2: ugh, which tiles are ENCLOSED by the loop???**

```
..........
.S------7.
.|F----7|.
.||OOOO||.
.||OOOO||.
.|L-7F-J|.
.|II||II|.
.L--JL--J.
..........
```

In this example *I* tiles are enclosed but *0* tiles aren't.

Thoughts:

- There needs to be EXACTLY ONE PIECE OF PIPE LOOP in each direction from a point (any dist away) for it to be considered enclosed
- If we mark the area that is part of our PIPE LOOP, and the rest of the area as false, then we can check all the 'false' areas, if they are enclosed change to true, and then sum the trues.

In [9]:
def is_enclosed(location: tuple[int, int], pipe_loop_map: dict, y_max: int, x_max: int) -> bool:
    loc_y, loc_x = location
    # check up
    above_left_bends_or_flats = 0
    for y_idx in range(loc_y):
        # if we just check for left bends or flats
        if pipe_loop_map[(y_idx, loc_x)] in ("7", "J", "-"):
            above_left_bends_or_flats += 1
    if above_left_bends_or_flats % 2 == 0:
        return False
    
    # check down
    below_left_bends_or_flats = 0
    for y_idx in range(loc_y+1, y_max+1):
        # if we just check for left bends or flats
        if pipe_loop_map[(y_idx, loc_x)] in ("7", "J", "-"):
            below_left_bends_or_flats += 1
    if below_left_bends_or_flats % 2 == 0:
        return False
    
    # check left
    left_up_bends_or_straights = 0
    for x_idx in range(loc_x):
        # if we just check for left bends or flats
        if pipe_loop_map[(loc_y, x_idx)] in ("L", "J", "|"):
            left_up_bends_or_straights += 1
    if left_up_bends_or_straights % 2 == 0:
        return False

    # check right
    right_up_bends_or_straights = 0
    for x_idx in range(loc_x+1, x_max+1):
        # if we just check for left bends or flats
        if pipe_loop_map[(loc_y, x_idx)] in ("L", "J", "|"):
            right_up_bends_or_straights += 1
    if right_up_bends_or_straights % 2 == 0:
        return False

    return True

In [10]:
with open("./example-4-enclosed.txt") as f:
    example_4_enclosed = [line.strip() for line in f.readlines()]
with open("./example-8-enclosed.txt") as f:
    example_8_enclosed = [line.strip() for line in f.readlines()]

In [11]:
def replace_S_properly(S_loc: tuple[int, int], grid_map: dict) -> str:
    """
    returns what S in pipe form would be
    """
    L_posib = set()
    R_posib = set()
    U_posib = set()
    D_posib = set()
    S_y, S_x = S_loc

    # Left
    if item := grid_map.get((S_y, S_x-1)):
        if item in ("-", "L", "F"):
            L_posib.update(
                ("-", "J", "7")
            )
    # Right
    if item := grid_map.get((S_y, S_x+1)):
        if item in ("-", "7", "J"):
            R_posib.update(
                ("-", "F", "L")
            )
    # Up
    if item := grid_map.get((S_y-1, S_x)):
        if item in ("|", "7", "F"):
            U_posib.update(
                ("|", "J", "L")
            )
    # Down
    if item := grid_map.get((S_y+1, S_x)):
        if item in ("|", "J", "L"):
            D_posib.update(
                ("|", "7", "F")
            )
    
    posib_sets = [L_posib, R_posib, U_posib, D_posib]
    posib = set(("-", "L", "F", "|", "J", "7"))
    for posib_set in posib_sets:
        if posib_set:
            posib.intersection_update(posib_set)

    assert len(posib) == 1

    return list(posib)[0]

In [12]:
def part2(lines: list[str]):
    grid_map, starting_position_tuple = create_grid_map(lines)
    starting_position = Vector2D(y=starting_position_tuple[0], x=starting_position_tuple[1])


    pipe_loop_map = {
        key: 0 for key in grid_map.keys()
    }


    # INITIAL SET UP
    first_moves_list = find_first_moves(grid_map, starting_position)
    move1, dir1 = first_moves_list[0]
    plumber1 = Plumber(starting_position, move1, grid_map)
    plumber1.move(move1)
    plumber1.current_dir = dir1

    pipe_loop_map[starting_position.as_tuple] = grid_map[plumber1.pos.as_tuple]
    pipe_loop_map[plumber1.pos.as_tuple] = grid_map[plumber1.pos.as_tuple]

    steps_plumber_1 = 1
    # REST OF THE LOOP!
    while True:
        plumber1.move()
        pipe_loop_map[plumber1.pos.as_tuple] = grid_map[plumber1.pos.as_tuple]
        steps_plumber_1 += 1
        # print(plumber1.pos, grid_map[plumber1.pos.as_tuple], plumber1.current_dir)
        if grid_map[plumber1.pos.as_tuple] == "S":
            break
    
    non_pipe_locations = [key for key,val in pipe_loop_map.items() if not val]

    y_max = len(lines) - 1
    x_max = len(lines[0]) - 1

    # Replace S with pipe now
    new_S = replace_S_properly(starting_position.as_tuple, grid_map)
    pipe_loop_map[starting_position.as_tuple] = new_S

    enclosed_bits = 0
    for location in non_pipe_locations:
        if is_enclosed(location, pipe_loop_map, y_max, x_max):
            enclosed_bits += 1
    
    return enclosed_bits

assert part2(example_4_lines) == 1
assert part2(example_4_enclosed) == 4
assert part2(example_8_enclosed) == 8
part2(input_lines)

265